In [79]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import re

import pandas as pd

In [105]:
def driver_init():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    # options.add_argument('--headless')
    driver = webdriver.Chrome()
    driver.set_window_size(1920,1080)
    return driver

def element_style_url(string):
    string = 'background-position: center center; background-size: contain; background-image: url("https://images.lider.cl/wmtcl?source=url[file:/productos/4493165a.jpg]&sink");'

    # Utilizamos una expresión regular para buscar el patrón del URL
    pattern = r'url\("([^"]+)"\)'
    match = re.search(pattern, string)

    if match:
        url = match.group(1)
    else:
        print("No se encontró ninguna URL en el string.")
    return url 

In [86]:
#Scrapp pdp lider.cl

In [106]:
def scrapp_pdp(sku):
    url = f'https://www.lider.cl/supermercado/product/sku/{sku}'
    driver = driver_init()
    driver.get(url)
    WebDriverWait(driver=driver, timeout=30).until(visibility_of_element_located((By.CLASS_NAME, 'product-detail-card__buttons-container')))
    content = driver.page_source
    driver.close()
    soup = BeautifulSoup(content)
    product_img_url = element_style_url(soup.find('figure', attrs={'class':'desktop__primary-image-container'})['style' ]).replace('&amp;','&')
    data = {'sku':sku, 
            'url':url,
            'product_img_url':product_img_url
            }
    df = pd.json_normalize(data)
    return df

In [107]:
sku=4493165
df = scrapp_pdp(sku)

In [117]:
def generate_html_with_images(df, output_file):
    html = "<html><head></head><body><table>"
    html += "<tr><th>{}</th><th>{}</th></tr>".format(df.columns[0], df.columns[1], df.columns[2])
    for i in range(len(df)):
        row = df.iloc[i]
        html += "<tr><td>{}</td><td><img src='{}'></td></tr>".format(row[df.columns[0]], row[df.columns[1]], df.columns[2])
    html += "</table></body></html>"

    with open(output_file, 'w') as file:
        file.write(html)

In [120]:
df

,sku,url,product_img_url
0,4493165,https://www.lider.cl/supermercado/product/sku/...,https://images.lider.cl/wmtcl?source=url[file:...


In [140]:
def generate_html_with_images(df,output_file):
    html = "<html><head></head><body><table>"
    html += f'''
        <tr>
            <th>{df.columns[0]}</th>
            <th>{df.columns[1]}</th>
            <th>{df.columns[2]}</th>
        </tr>
        '''
    for i in range(len(df)):
        row = df.iloc[i]
        html += '''
            <tr>
                <td>{}</td>
                <td>{}</td>
                <td>
                    <img src={}>
                </td>
            </tr>'''.format(row[df.columns[0]], row[df.columns[1]],row[df.columns[2]])
        html += "</table></body></html>"
    with open(output_file, 'w') as file:
        file.write(html)
    return html

In [141]:
df

,sku,url,product_img_url
0,4493165,https://www.lider.cl/supermercado/product/sku/...,https://images.lider.cl/wmtcl?source=url[file:...


In [143]:
# Generar el archivo HTML
output_file = 'output.html'
print(generate_html_with_images(df, output_file))

<html><head></head><body><table>
        <tr>
            <th>sku</th>
            <th>url</th>
            <th>product_img_url</th>
        </tr>
        
            <tr>
                <td>4493165</td>
                <td>https://www.lider.cl/supermercado/product/sku/4493165</td>
                <td>
                    <img src=https://images.lider.cl/wmtcl?source=url[file:/productos/4493165a.jpg]&sink>
                </td>
            </tr></table></body></html>
